In [1]:
import pandas as pd
import numpy as np
import sys
import random
import os
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-08-16 15:58:02.046038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 15:58:02.125209: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-16 15:58:02.127183: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-16 15:58:02.127191: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [4]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [11]:
def lstm(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.LSTM(50),
#     tf.keras.layers.Dense(1)
# ])
    
    input_layer = Input(shape=(X_train.shape[1],))
    reshaped_input = tf.expand_dims(input_layer, axis=1)
    
    lstm_layer1 = LSTM(units=50, return_sequences=True)(reshaped_input)
    dropout_layer1 = Dropout(dropout)(lstm_layer1)

    lstm_layer2 = LSTM(units=50)(dropout_layer1)
    dropout_layer2 = Dropout(dropout)(lstm_layer2)
    
    output_layer = Dense(units=1)(dropout_layer2)

    model = Model(inputs=input_layer, outputs=output_layer) 

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    checkpoint = ModelCheckpoint('../models/lstm_checkpoint.h5', monitor='val_loss', save_best_only=True,
                                save_weights_only=False)
    early_stopping = EarlyStopping(monitor='val_loss', patience=300)
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_split=0.2, 
              callbacks=[checkpoint, early_stopping])
    
#     model = keras.Sequential()
#     model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(LSTM(50, return_sequences=True, input_shape=(window_size, x_train.shape[-1])))
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM((window_size * 2), return_sequences=True))) 
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM(window_size, return_sequences=False))) 
#     model.add(Dense(units=1))
#     return model
    best_model = keras.models.load_model('../models/lstm_checkpoint.h5')
    return best_model

In [7]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

In [8]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = lstm(X_train, y_train)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

#### delete from here

In [12]:
idx=0
device_indices = final_df[final_df.latitude==latitudes[idx]].index
device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
test_df = final_df.loc[device_indices]
train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)

In [13]:
# %%timeit
model = lstm(X_train, y_train)

Epoch 1/1000
289/289 [==============================] - 4s 5ms/step - loss: 695.5039 - val_loss: 846.3549
Epoch 2/1000
289/289 [==============================] - 1s 3ms/step - loss: 431.6653 - val_loss: 681.6651
Epoch 3/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.0635 - val_loss: 672.1375
Epoch 4/1000
289/289 [==============================] - 1s 3ms/step - loss: 416.2322 - val_loss: 669.9920
Epoch 5/1000
289/289 [==============================] - 1s 3ms/step - loss: 415.4362 - val_loss: 673.6865
Epoch 6/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.3436 - val_loss: 672.8229
Epoch 7/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.5896 - val_loss: 669.1880
Epoch 8/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.1212 - val_loss: 671.3693
Epoch 9/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.8275 - val_loss: 671.7575
Epoch 10/1000
289/289 [=======================

289/289 [==============================] - 1s 3ms/step - loss: 414.7778 - val_loss: 671.7044
Epoch 78/1000
289/289 [==============================] - 1s 3ms/step - loss: 415.3608 - val_loss: 671.5324
Epoch 79/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.9089 - val_loss: 671.7939
Epoch 80/1000
289/289 [==============================] - 1s 3ms/step - loss: 415.3897 - val_loss: 674.3209
Epoch 81/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.2139 - val_loss: 670.6810
Epoch 82/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.6636 - val_loss: 668.2909
Epoch 83/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.6742 - val_loss: 671.3555
Epoch 84/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.5464 - val_loss: 671.6293
Epoch 85/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.4171 - val_loss: 670.7085
Epoch 86/1000
289/289 [============================

289/289 [==============================] - 1s 3ms/step - loss: 414.8315 - val_loss: 670.8149
Epoch 154/1000
289/289 [==============================] - 1s 3ms/step - loss: 415.2170 - val_loss: 670.7731
Epoch 155/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.7069 - val_loss: 670.9807
Epoch 156/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.9243 - val_loss: 671.0523
Epoch 157/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.9384 - val_loss: 673.9691
Epoch 158/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.8759 - val_loss: 672.9713
Epoch 159/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.0920 - val_loss: 673.4284
Epoch 160/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.6152 - val_loss: 672.5039
Epoch 161/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.3233 - val_loss: 670.3912
Epoch 162/1000
289/289 [===================

Epoch 229/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.9459 - val_loss: 669.3738
Epoch 230/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.3002 - val_loss: 665.2392
Epoch 231/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.3773 - val_loss: 669.1031
Epoch 232/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.2934 - val_loss: 661.4808
Epoch 233/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.4930 - val_loss: 667.5723
Epoch 234/1000
289/289 [==============================] - 1s 3ms/step - loss: 415.2413 - val_loss: 667.7667
Epoch 235/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.3087 - val_loss: 670.2642
Epoch 236/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.1546 - val_loss: 663.3553
Epoch 237/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.8571 - val_loss: 665.4574
Epoch 238/1000
289/289 [====

Epoch 305/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.5689 - val_loss: 674.0450
Epoch 306/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.7290 - val_loss: 667.2794
Epoch 307/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.2067 - val_loss: 674.3840
Epoch 308/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.2674 - val_loss: 662.5245
Epoch 309/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.7274 - val_loss: 671.5217
Epoch 310/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.5219 - val_loss: 674.9957
Epoch 311/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.9856 - val_loss: 671.9617
Epoch 312/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.4479 - val_loss: 662.8620
Epoch 313/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.9115 - val_loss: 663.2092
Epoch 314/1000
289/289 [====

...

Epoch 685/1000
289/289 [==============================] - 1s 3ms/step - loss: 415.2518 - val_loss: 660.8890
Epoch 686/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.6500 - val_loss: 655.8054
Epoch 687/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.3695 - val_loss: 659.3856
Epoch 688/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.2730 - val_loss: 657.5309
Epoch 689/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.7261 - val_loss: 656.5163
Epoch 690/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.8763 - val_loss: 663.9705
Epoch 691/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.0525 - val_loss: 666.0038
Epoch 692/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.5498 - val_loss: 651.1341
Epoch 693/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.1840 - val_loss: 654.4285
Epoch 694/1000
289/289 [====

Epoch 761/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.8449 - val_loss: 657.4319
Epoch 762/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.8128 - val_loss: 659.0154
Epoch 763/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.3176 - val_loss: 657.4855
Epoch 764/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.5842 - val_loss: 649.6936
Epoch 765/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.5333 - val_loss: 655.3505
Epoch 766/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.3105 - val_loss: 658.1813
Epoch 767/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.5615 - val_loss: 654.8948
Epoch 768/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.8434 - val_loss: 655.7292
Epoch 769/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.0048 - val_loss: 653.4807
Epoch 770/1000
289/289 [====

Epoch 837/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.7674 - val_loss: 653.1145
Epoch 838/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.0606 - val_loss: 655.6910
Epoch 839/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.8310 - val_loss: 658.1832
Epoch 840/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.7202 - val_loss: 658.8815
Epoch 841/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.2584 - val_loss: 649.1756
Epoch 842/1000
289/289 [==============================] - 1s 3ms/step - loss: 414.0912 - val_loss: 651.6248
Epoch 843/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.0686 - val_loss: 654.6999
Epoch 844/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.2412 - val_loss: 650.0855
Epoch 845/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.7339 - val_loss: 655.8470
Epoch 846/1000
289/289 [====

Epoch 913/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.7024 - val_loss: 656.4092
Epoch 914/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.8049 - val_loss: 654.4705
Epoch 915/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.5248 - val_loss: 656.2897
Epoch 916/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.3841 - val_loss: 658.9722
Epoch 917/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.2159 - val_loss: 660.3853
Epoch 918/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.3155 - val_loss: 651.7358
Epoch 919/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.1921 - val_loss: 645.7986
Epoch 920/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.9416 - val_loss: 656.7823
Epoch 921/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.0629 - val_loss: 654.1283
Epoch 922/1000
289/289 [====

Epoch 989/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.9459 - val_loss: 657.9404
Epoch 990/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.7133 - val_loss: 657.9167
Epoch 991/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.4037 - val_loss: 645.0582
Epoch 992/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.9020 - val_loss: 655.9874
Epoch 993/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.2944 - val_loss: 661.9711
Epoch 994/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.4375 - val_loss: 657.0278
Epoch 995/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.4096 - val_loss: 656.4325
Epoch 996/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.7933 - val_loss: 650.0142
Epoch 997/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.3850 - val_loss: 661.7311
Epoch 998/1000
289/289 [====

In [14]:
%%timeit
y_pred = model.predict(X_test) 

66/66 [==============================] - 0s 1ms/step
113 ms ± 6.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### end here

In [ ]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    rmse, mape = cross_validation(final_df, i)
    rmse_list.append(rmse)
    mape_list.append(mape)
    print(f'{device_ids[i]} successful')

In [ ]:
mean_rmse, mean_mape = np.mean(rmse_list), np.mean(mape_list)          
mean_rmse, mean_mape

In [ ]:
rmse_list

In [ ]:
mape_list